本节讨论python中的序列类型

In [1]:
# python中的序列有三种：列表类，元组类，字符串类
# python默认使用的是unicode字符，一个字符占据2个字节
# 询问序列的时间我们认为是O(1)

In [3]:
# 引用数组
patients = ['jojo', 'dio', 'killer_queen']
# 这个列表中，每个数组储存的是每个字符串类的地址
# 类似于：
patient = [['j', 'o', 'j', '0'], [], []]
# 当然可以写一个实例

In [ ]:
# https://blog.csdn.net/as480133937/article/details/87305247 关于浅拷贝和深拷贝，按住ctrl键可以直接到达

紧凑数组

In [21]:
import sys
sys.getsizeof([1]), sys.getsizeof([])
# 一个列表占据56字节，每添加一个指针则多8字节

(64, 56)

In [38]:
a = 1
sys.getsizeof(a), sys.getsizeof(id(a)), hex(id(a))
# 发现，一个整数变量占据28字节，如果产生一个指针，便又花费32字节

(28, 32, '0x7ff9f17a29b8')

In [43]:
l2 = [a]
sys.getsizeof(l2), sys.getsizeof(id(l2[0])) , hex(id(l2)), hex(id(a))
# 如果我们还是使用列表来储存数据，占据了28 + 64 + 32的内存,远远扩张内存占据

(64, 32, '0x24de921a140', '0x7ff9f17a29b8')

In [ ]:
# https://blog.csdn.net/qq_43657442/article/details/108431857 看这个更能理解

In [49]:
# 为了减小内存占据，我们使用数组来储存数据
from array import array
prims = array('i', [1, 2, 3])
prims # 其中每个数据只占有2字节

array('i', [1, 2, 3])

In [ ]:
# https://www.cainiaojc.com/python/python-array.html 数组文章，当然一般不建议使用这个，计算包中都有类似的计算数组

动态数组和摊销

In [56]:
# 在python列表当中，我们是动态的申请内存的
import sys
data = []
for i in range(20):
    a = len(data)
    b = sys.getsizeof(data)
    print('length: {0:3d}; size: {1:3d}'.format(a, b))
    data.append(None)

length:   0; size:  56
length:   1; size:  88
length:   2; size:  88
length:   3; size:  88
length:   4; size:  88
length:   5; size: 120
length:   6; size: 120
length:   7; size: 120
length:   8; size: 120
length:   9; size: 184
length:  10; size: 184
length:  11; size: 184
length:  12; size: 184
length:  13; size: 184
length:  14; size: 184
length:  15; size: 184
length:  16; size: 184
length:  17; size: 248
length:  18; size: 248
length:  19; size: 248


In [55]:
# 发现内存不是随着元素的增加而增加
# list是预先准备好多余的内存，方便你去添加删减元素，等内存不够了再去申请多的内存
# 原因是python是用c语言写出来的

实现动态数组

In [53]:
import ctypes

class DynamicArray:
    """实现动态数组的编写，类似list"""
    def __init__(self):
        """创建一个空数组"""
        self._n = 0
        self._capacity = 1
        self._A = self._make_array(self._capacity)

    def __len__(self):
        """返回数组中的元素个数"""
        return self._n

    def __getitem__(self, k):
        """实现访问功能"""
        if k < 0 or k >= self._n:
            raise IndexError('index out of range')
        return self._A[k]

    def append(self, obj):
        """在数组末尾添加元素"""
        if self._n == self._capacity:
            self._resize(2 * self._capacity)
        self._A[self._n] = obj
        self._n += 1

    def _resize(self, capacity):
        """扩增数组"""
        B = self._make_array(capacity)
        for i in range(self._n):
            B[i] = self._A[i]
        self._A = B
        self_capacity = capacity

    def _make_array(self, capacity):
        """返回新的数组"""
        return (capacity * ctypes.py_object)() # 返回python的对象

# 这里实现一个动态的数组编程
# ctypes不需要了解，知道是什么就行了
# https://zhuanlan.zhihu.com/p/18591487058
# 在上述认知之下，其实python的底层代码实现还是运用的c语言
# 所以，对列表进行插入，排序，增添，其实是最占据时间的步骤

总结

In [ ]:
# 本节主要讲述内存分布以及列表原理以及数组实操
# 练手 ：凯撒密码
# 把英文字母用后n位的字母替代，例如A->D, B->E,
# 编写一个类，不要求动态分布内存，实现编码以及解码的功能

In [63]:
class CaesarCode:
    """凯撒密码"""
    def __init__(self, shift):
        """创建编码表"""
        encoder = [None] * 26
        decoder = [None] * 26
        for i in range(26):
            encoder[i] = chr(ord('a') + (i + shift) % 26)       # ord函数返回是在unicode的编码
            decoder[i] = chr(ord('a') + (i - shift) % 26)

        self._forward = ''.join(encoder)
        self._backward = ''.join(decoder)

    def encode(self, text):
        """给问文本加密"""
        return self._transform(text, self._forward)

    def decode(self, text):
        """给文本解码"""
        return self._transform(text, self._backward)

    def _transform(self, text, transform):
        l = len(text)
        code = list(text)
        for i in range(l):
            code[i] = transform[ord(text[i]) - ord('a')]
        return ''.join(code)

if __name__ == '__main__':
    meg = CaesarCode(1)
    code = meg.encode('hello')
    print(code)
    answer = meg.decode(code)
    print(answer)

# 当然有很大的瑕疵，比如大写字母的问题
# 或者能跟进一步输入密文再次加密然后解码

ifmmp
hello
